artist.json.gzは，オープンな音楽データベースMusicBrainzの中で，アーティストに関するものをJSON形式に変換し，gzip形式で圧縮したファイルである．

このファイルには，1アーティストに関する情報が1行にJSON形式で格納されている．

JSON形式の概要は以下の通りである．

artist.json.gzのデータをKey-Value-Store (KVS) およびドキュメント志向型データベースに格納・検索することを考える．

KVSとしては，LevelDB，Redis，KyotoCabinet等を用いよ．

ドキュメント志向型データベースとして，MongoDBを採用したが，CouchDBやRethinkDB等を用いてもよい．

** 0. KVSの構築 **

Key-Value-Store (KVS) を用い，アーティスト名（name）から活動場所（area）を検索するためのデータベースを構築せよ．

In [1]:
import gzip
import json

*** leveldbパッケージを使った場合 ***

In [2]:
fname = 'nlp100data/artist.json.gz'
fname_db = 'nlp100data/test_db'

In [1]:
import leveldb

In [3]:
db = leveldb.LevelDB(fname_db)

In [25]:
with gzip.open(fname, 'rt') as data_file:
    for i , line in enumerate(data_file):
        if i > -1:
            run_dict = json.loads(line)
            key = "{}\t{}".format(run_dict['name'] , run_dict['id'])
            #print(key)
            value = run_dict.get("area" , "")
            db.Put(key.encode() , value.encode())

            #print("[{}] {}".format(value ,key))

In [27]:
Ndata = len(list(db.RangeIter(include_value=False)))

In [28]:
print("registered {} pairs".format(Ndata))

registered 921337 pairs


*** plyvelパッケージを使った場合***

https://plyvel.readthedocs.io/en/latest/user.html#getting-started

http://kuroneko0208.hatenablog.com/entry/2014/10/06/090128

In [5]:
fname = 'nlp100data/artist.json.gz'
fname_db = 'nlp100data/test2_db'

In [6]:
import plyvel

In [8]:
db = plyvel.DB(fname_db , create_if_missing=True)

In [9]:
with gzip.open(fname, 'rt') as data_file:
    for i , line in enumerate(data_file):
        if i > -1:
            run_dict = json.loads(line)
            key = "{}\t{}".format(run_dict['name'] , run_dict['id'])
            #print(key)
            value = run_dict.get("area" , "")
            db.put(key.encode() , value.encode())

            #print("[{}] {}".format(value ,key))

In [10]:
len([key for key , value in db])

921337

In [11]:
db.close()

#### KVSの検索

60で構築したデータベースを用い，特定の（指定された）アーティストの活動場所を取得せよ．

In [13]:
db = plyvel.DB(fname_db , create_if_missing=True)

In [27]:
["{} [{}]".format(key_value[0] , key_value[1]) for i , key_value in enumerate(db) if i < 10]

["b'! Obtain?\\t909918' [b'']",
 "b'!!\\t678425' [b'']",
 "b'!!!\\t109013' [b'United States']",
 "b'!!Swanhunter\\t265567' [b'']",
 "b'!((0rkza1\\t629789' [b'']",
 "b'!ASQ!\\t420970' [b'']",
 "b'!ATTENTION!\\t617688' [b'']",
 "b'!Action Pact!\\t95246' [b'United Kingdom']",
 "b'!BNB! All-Stars\\t171627' [b'']",
 "b'!Bang Elektronika\\t172294' [b'Canada']"]

In [36]:
sample_names = [key_value[0] for i , key_value in enumerate(db) if i < 100]

In [40]:
sample_names[70].decode()

'"INFINITY 16,SWEET SOP"\t1155771'

In [ ]:
sample_names[70].decode

In [33]:
db.get(sample_names[7])

b'United Kingdom'

In [25]:
db.get(b"!ATTENTION!\t617688")

b''

In [26]:
db.get(b'!Action Pact!\\t95246')

In [28]:
db.get(b'!Bang Elektronika\\t172294')

In [12]:
%%bash
git commit -a -m "60"

[master 08c8b42] 60
 1 file changed, 161 insertions(+), 7 deletions(-)
